In [5]:
import pandas as pd
# Загружаем данные
file = 'S06-hw-dataset-01.csv'
data = pd.read_csv(file)

# Первичный анализ данных
print("Типы данных и информация о пропусках:")
data.info()
print("\nПервые строки данных:")
print(data.head())
print("\nОсновные статистики по числовым столбцам:")
print(data.describe())

# Проверка распределения целевой переменной
target_variable = data['target'].value_counts(normalize=True)
print("\nРаспределение целевой переменной:")
print(target_variable)

Типы данных и информация о пропусках:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             12000 non-null  int64  
 1   num01          12000 non-null  float64
 2   num02          12000 non-null  float64
 3   num03          12000 non-null  float64
 4   num04          12000 non-null  float64
 5   num05          12000 non-null  float64
 6   num06          12000 non-null  float64
 7   num07          12000 non-null  float64
 8   num08          12000 non-null  float64
 9   num09          12000 non-null  float64
 10  num10          12000 non-null  float64
 11  num11          12000 non-null  float64
 12  num12          12000 non-null  float64
 13  num13          12000 non-null  float64
 14  num14          12000 non-null  float64
 15  num15          12000 non-null  float64
 16  num16          12000 non-null  float64
 17  num17       

In [6]:
from sklearn.model_selection import train_test_split
# Определение признаков и целевой переменны
X = data.drop(columns=['target', 'id'])  # исключение 'target' и 'id'
y = data['target']
# Разделение данных на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (9600, 28)
Размер тестовой выборки: (2400, 28)


In [7]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
# Создание Dummy классификатора
dummy_classifier = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_classifier.fit(X_train, y_train)
# Создание логистической регрессии с пайплайном
logistic_regression_model = Pipeline([
    ('scaler', StandardScaler()), 
    ('logreg', LogisticRegression(random_state=42))
])
logistic_regression_model.fit(X_train, y_train)
# Оценка моделей на тестовой выборке
dummy_predictions = dummy_classifier.predict(X_test)
logreg_predictions = logistic_regression_model.predict(X_test)
# Метрики для Dummy классификатора
dummy_accuracy = accuracy_score(y_test, dummy_predictions)
dummy_f1_score = f1_score(y_test, dummy_predictions)
dummy_roc_auc = roc_auc_score(y_test, dummy_classifier.predict_proba(X_test)[:, 1])
# Метрики для логистической регрессии
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
logreg_f1_score = f1_score(y_test, logreg_predictions)
logreg_roc_auc = roc_auc_score(y_test, logistic_regression_model.predict_proba(X_test)[:, 1])
# Выводим результаты
print("Метрики для Dummy классификатора:")
print(f"Accuracy: {dummy_accuracy}, F1: {dummy_f1_score}, ROC-AUC: {dummy_roc_auc}")
print("\nМетрики для логистической регрессии:")
print(f"Accuracy: {logreg_accuracy}, F1: {logreg_f1_score}, ROC-AUC: {logreg_roc_auc}")

Метрики для Dummy классификатора:
Accuracy: 0.6766666666666666, F1: 0.0, ROC-AUC: 0.5

Метрики для логистической регрессии:
Accuracy: 0.8275, F1: 0.7076271186440678, ROC-AUC: 0.8746905312071505


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
# Инициализация моделей
dtc_clf = DecisionTreeClassifier(random_state=42)
rfc_clf = RandomForestClassifier(random_state=42)
ada_clf = AdaBoostClassifier(random_state=42)
# Подбор гиперпараметров для DecisionTreeClassifier
dtc_param_grid = {'max_depth': [3, 5, 10, None], 'min_samples_leaf': [1, 2, 4]}
dtc_grid_search = GridSearchCV(dtc_clf, dtc_param_grid, cv=5, n_jobs=-1)
dtc_grid_search.fit(X_train, y_train)
# Подбор гиперпараметров для RandomForestClassifier
rfc_param_grid = {'max_depth': [3, 5, 10, None], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 'log2']}
rfc_grid_search = GridSearchCV(rfc_clf, rfc_param_grid, cv=5, n_jobs=-1)
rfc_grid_search.fit(X_train, y_train)
# Подбор гиперпараметров для AdaBoostClassifier
ada_param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1]}
ada_grid_search = GridSearchCV(ada_clf, ada_param_grid, cv=5, n_jobs=-1)
ada_grid_search.fit(X_train, y_train)
# Оценка лучших моделей на тестовой выборке
dtc_best_pred = dtc_grid_search.best_estimator_.predict(X_test)
rfc_best_pred = rfc_grid_search.best_estimator_.predict(X_test)
ada_best_pred = ada_grid_search.best_estimator_.predict(X_test)
# Метрики для моделей
# для DecisionTreeClassifier
dtc_accuracy = accuracy_score(y_test, dtc_best_pred)
dtc_f1 = f1_score(y_test, dtc_best_pred)
dtc_roc_auc = roc_auc_score(y_test, dtc_grid_search.best_estimator_.predict_proba(X_test)[:, 1])
# для RandomForestClassifier
rfc_accuracy = accuracy_score(y_test, rfc_best_pred)
rfc_f1 = f1_score(y_test, rfc_best_pred)
rfc_roc_auc = roc_auc_score(y_test, rfc_grid_search.best_estimator_.predict_proba(X_test)[:, 1])
# для AdaBoostClassifier
ada_accuracy = accuracy_score(y_test, ada_best_pred)
ada_f1 = f1_score(y_test, ada_best_pred)
ada_roc_auc = roc_auc_score(y_test, ada_grid_search.best_estimator_.predict_proba(X_test)[:, 1])
# Выводим результаты
print("\nМетрики для DecisionTreeClassifier:")
print(f"Accuracy: {dtc_accuracy}, F1: {dtc_f1}, ROC-AUC: {dtc_roc_auc}")
print("\nМетрики для RandomForestClassifier:")
print(f"Accuracy: {rfc_accuracy}, F1: {rfc_f1}, ROC-AUC: {rfc_roc_auc}")
print("\nМетрики для AdaBoostClassifier:")
print(f"Accuracy: {ada_accuracy}, F1: {ada_f1}, ROC-AUC: {ada_roc_auc}")


Метрики для DecisionTreeClassifier:
Accuracy: 0.8679166666666667, F1: 0.783617747440273, ROC-AUC: 0.8787560782844954

Метрики для RandomForestClassifier:
Accuracy: 0.9258333333333333, F1: 0.8792401628222524, ROC-AUC: 0.9665757833528008

Метрики для AdaBoostClassifier:
Accuracy: 0.8408333333333333, F1: 0.7336122733612274, ROC-AUC: 0.9018087260677466


In [13]:
import joblib
import json
# Сохранение моделей
joblib.dump(dtc_grid_search.best_estimator_, 'model.joblib')
# Сохранение метрик и гиперпараметров
metrics = {
    'DecisionTree': {'accuracy': dtc_accuracy, 'f1': dtc_f1, 'roc_auc': dtc_roc_auc},
    'RandomForest': {'accuracy': rfc_accuracy, 'f1': rfc_f1, 'roc_auc': rfc_roc_auc},
    'AdaBoost': {'accuracy': ada_accuracy, 'f1': ada_f1, 'roc_auc': ada_roc_auc},
}
with open('metrics_test.json', 'w') as f: # записывание данных
    json.dump(metrics, f)
# Сохранение метаданных о модели
model_meta = { # сохранение данных
    'model': 'DecisionTreeClassifier',
    'parameters': dtc_grid_search.best_params_,
    'metrics': metrics['DecisionTree']
}
with open('model_meta.json', 'w') as f: # сохранение моделей
    json.dump(model_meta, f)